# Trabalho 1 Introdução a Ciência de Dados

  O seguinte código realiza webScraping ao site https://www.usadosbr.com/carros/br,
  utilizando a biblioteca BeautifulSoup para coletar os dados e a biblioteca pandas para montar o DataFrame.

## Instalando e importando bibliotecas que serão utilizadas

In [ ]:
try:
  from bs4 import BeautifulSoup
  import requests
except:
  !pip install BeautifulSoup4

In [ ]:
import pandas as pd

In [ ]:
import re

In [ ]:
from tqdm import tqdm

## Declarações

In [ ]:
caracteristicas = { #  Criando dicionário que será
    0:[],           #utilizado para criar o DataFrame.
    1:[],
    2:[],
    3:[],
    4:[],
    5:[],
    6:[],
    7:[],
    8:[]
}
#  As informações acerca da marca, modelo, motor, preço e ano
#de cada carro são sempre apresentadas do respectivo código
#da lista abaixo.
#            marca         modelo         motor          preço         ano
codigos = ["css-8i3pvs", "css-jnq6z6", "css-kufh1x", "css-cn4x9d", "css-ljtdvh"]

In [ ]:
'''
  Método responsável por capturar a String
referente à característica do carro.
  Recebe como parâmetros o ínicio, que é a
posição em que a String começa, o texto, e
um caractere, que será o sinal de que a
String desejada acabou.
'''
def getNome(inicio, texto, char) -> str:
    nome = ''
    #  Adiciona caractere por caractere na
    #variável nome até o caractere de parada.
    for caracter in texto[inicio : ]:
        if caracter == char: break
        nome += caracter
    return nome

## Execução do Scraping

In [ ]:
for pagina in tqdm(range(1501, 3501)):
  url = f"https://www.usadosbr.com/carros/br?page={pagina}"
  response = requests.get(url)
  content = response.content
  soup = BeautifulSoup(content, "html.parser")
  '''
    As informações dos carros estão na última linha
  do código html, então a variável texte recebe o
  código html como String, mas dividido por '\n',
  que é quando se pula uma linha, logo em seguida
  é atribuída à variável texto apenas a última linha.
  '''
  texto = str(soup).split('\n')
  texto = texto[-1]

  #  O laço vai até 20 pois em cada
  #página do site há apenas 20 carros.
  for i in range(0, 20):
    for i, codigo in enumerate(codigos):
      nome = ''
      '''
        O código em si aparece várias vezes no
      html, porém quando é apresentado após um
      'class="' é para apresentar a informação
      de um carro.
      '''
      codigo = f'class="{codigo}">'
      if i < 4:
        #  O início de onde a String será encontrada
        #é dado pelo index() do codigo + o seu tamanho.
        inicio = texto.index(codigo) + len(codigo)
        nome = getNome(inicio, texto, '<')
        if i == 2:
          #  Pegando o câmbio
          if 'Manual' in nome:
            nome = nome[:-7]
            caracteristicas[7].append('Manual')
          elif 'Automático' in nome:
            nome = nome[:-11]
            caracteristicas[7].append('Automático')
          elif 'Semi-automático' in nome:
            nome = nome[:-15]
            caracteristicas[7].append('Semi-automático')
          elif 'CVT' in nome:
            nome = nome[:-4]
            caracteristicas[7].append('CVT')
          else:
            caracteristicas[7].append('N/A')
          #  Pegando o combustível
          if 'Á/G' in nome:
            nome = nome[4:]
            caracteristicas[8].append('Á/G')
          elif 'Diesel' in nome:
            nome = nome[7:]
            caracteristicas[8].append('Diesel')
          elif 'Gasolina' in nome:
            nome = nome[9:]
            caracteristicas[8].append('Gasolina')
          elif 'Híbrido' in nome:
            nome = nome[8:]
            caracteristicas[8].append('Híbrido')
          elif 'Elétrico' in nome:
            nome = nome[9:]
            caracteristicas[8].append('Elétrico')
          elif 'Álcool' in nome:
            nome = nome[7:]
            caracteristicas[8].append('Álcool')
          elif 'GNV' in nome:
            nome = nome[4:]
            caracteristicas[8].append('GNV')
          else:
            print(nome)
            caracteristicas[8].append('Não informado')

        if i == 3:
          #  O preço é capturado da forma "RS1.500.000",
          #então no caso de ser um preço que está sendo
          #capturado, é retirado o "R$" e os pontos.
          nome = int(nome.replace('R$', '').replace('.', ''))
        caracteristicas[i].append(nome)

      #  Os dados referentes à quilometragem é especial,
      #pois não tem código, ele apenas é apresentado após
      #o ano, então o ano é encontrado separadamente, e
      #após se coletar o ano, o inicio avança em 16, que
      #é onde se encontra o valor da quilometragem.

      else:
        inicio = texto.index(codigo) + len(codigo) + 3
        nome = getNome(inicio, texto, '<')
        nome = nome.split('/')
        caracteristicas[i].append(int(nome[0]))
        caracteristicas[i+1].append(int(nome[1]))
        #  Aqui começa a captura da quilometragem.
        inicio += 16
        nome = int(getNome(inicio, texto, 'K').replace('.', ''))
        caracteristicas[i + 2].append(nome)
        texto = texto[inicio + 20: ]




100%|██████████| 2000/2000 [2:02:35<00:00,  3.68s/it]


## Criação do DataFrame

In [ ]:
#Cria um DataFrame a partir do dicionário
df = pd.DataFrame(caracteristicas)
#Renomeia as colunas
df.rename(columns = {0:'Marca', 1:'Modelo', 2:'Motor', 3:'Preço', 4:'Ano Fabricação', 5:'Ano Modelo', 6:'Quilometragem', 7:'Câmbio', 8:'Combustível'}, inplace = True)

## DataFrame gerado

In [ ]:
df

,Marca,Modelo,Motor,Preço,Ano Fabricação,Ano Modelo,Quilometragem,Câmbio,Combustível
0,Peugeot,208,1.6 Like,72900,2023,2023,0,Manual,Á/G
1,Suzuki,Vitara,1.6 JLX,29900,1995,1995,210516,Manual,Gasolina
2,Mini,Cooper,1.6 One,59900,2013,2013,79586,Manual,Gasolina
3,Fiat,Toro,1.8 Freedom 16V,92900,2017,2018,71000,Automático,Á/G
4,Citroën,C3,1.6 XTR,22900,2006,2007,231000,Manual,Á/G
...,...,...,...,...,...,...,...,...,...
39995,Honda,Civic,2.0 LXR,79990,2015,2016,83931,Automático,Á/G
39996,Honda,Civic,2.0 LX,115900,2019,2020,40800,Automático,Á/G
39997,Fiat,Ducato,2.3 Cargo L,169900,2018,2018,106580,Manual,Diesel
39998,Fiat,Ducato,2.3 Cargo L,169900,2018,2018,106580,Manual,Diesel


In [ ]:
from google.colab import files

In [ ]:
#Cria um arquivo csv a partir do DataFrame gerado
df.to_csv("DFcarros.csv", index=False)
#Faz o download do arquivo
files.download('DFcarros.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>